In [2]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [14]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA="/home/spika/datasets/sonido/voz/asr_sms_21070201"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath=PATH_DATA):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.diseases=[]
        self.ages=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            print("n=",n)
            speaker,disease,age,number=n.split("-")
            self.speakers.append(speaker)
            self.diseases.append(disease)
            self.ages.append(age)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases)
        self.iDiseaseLabels=self.leDiseases.transform(self.diseases)
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages)
        self.iAgeLabels=self.leAges.transform(self.ages)
        print(len(self.diseases))
        print(self.diseases)
        print(self.ages)
        print(self.speakers)

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNames= 673
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/GRM-SMS-5-00002.wav
n= GRM-SMS-5-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/VVV-SMS-6-00006.wav
n= VVV-SMS-6-00006
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/LMD-N-7-00005.wav
n= LMD-N-7-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/PCR-SMS-15-00003.wav
n= PCR-SMS-15-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/CII-N-7-00004.wav
n= CII-N-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/CRO-N-8-00003.wav
n= CRO-N-8-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_sms_21070201/LEY-N-5-00004.wav
n= LEY-N-5-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/spika/datasets/sonido/voz/asr_

In [15]:
trainX, trainYdiseases, trainYspeakers, trainYages = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=max(trainYdiseases)+1
outputSpeakers_dim=max(trainYspeakers)+1
outputAges_dim=max(trainYages)+1

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape)

2 134 18
(673, 30, 20) (673, 2) (673, 134) (673, 18)


In [16]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,hiden_dim=128):
    input = Input(shape=(30,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    print(x.shape)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim)(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim)(x)
    sd= Dense(hiden_dim)(x)
    ad= Dense(hiden_dim)(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages])
    return model

In [17]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 30, 20)
(None, 28, 128)
(None, 13, 128)
(None, 6, 128)
(None, 128)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 20)]     0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 28, 128)      7808        input_2[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 28, 128)      0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 13, 128)      49280       dropout_4[0][0]                  
______

In [19]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat], batch_size=100,epochs=500,validation_split=0.2)

Epoch 1/500
6/6 [==============================] - 0s 45ms/step - loss: 7.6758 - disease_loss: 0.4725 - speaker_loss: 4.8711 - age_loss: 2.3322 - disease_accuracy: 0.8197 - speaker_accuracy: 0.0149 - age_accuracy: 0.1989 - val_loss: 7.8932 - val_disease_loss: 0.4571 - val_speaker_loss: 5.0320 - val_age_loss: 2.4041 - val_disease_accuracy: 0.8296 - val_speaker_accuracy: 0.0074 - val_age_accuracy: 0.1926
Epoch 2/500
6/6 [==============================] - 0s 31ms/step - loss: 7.6832 - disease_loss: 0.4803 - speaker_loss: 4.8693 - age_loss: 2.3335 - disease_accuracy: 0.8197 - speaker_accuracy: 0.0149 - age_accuracy: 0.2100 - val_loss: 7.8979 - val_disease_loss: 0.4571 - val_speaker_loss: 5.0360 - val_age_loss: 2.4049 - val_disease_accuracy: 0.8296 - val_speaker_accuracy: 0.0074 - val_age_accuracy: 0.1926
Epoch 3/500
6/6 [==============================] - 0s 33ms/step - loss: 7.6842 - disease_loss: 0.4698 - speaker_loss: 4.8754 - age_loss: 2.3389 - disease_accuracy: 0.8197 - speaker_accurac

In [20]:
print(trainX[5:7,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(trainX[5:6,:,:])
print(predDisease,predSpeaker,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(2, 30, 20)
[[0.99871397 0.00128603]] [[6.91762951e-04 8.06121912e-04 2.52933705e-06 9.02071751e-09
  7.08566077e-08 3.19729770e-05 3.64227817e-05 1.53398130e-08
  1.86324996e-05 2.89735276e-08 5.34891287e-09 2.37871980e-04
  1.09531308e-08 1.11577583e-06 8.53047799e-03 6.86438170e-07
  4.48425526e-05 5.91196738e-07 1.01622149e-07 1.29188775e-05
  2.35601094e-09 1.09535900e-08 4.67869121e-09 2.08737880e-08
  9.70919549e-01 9.87694762e-08 2.87913249e-10 6.55654997e-09
  6.24760119e-08 3.05420089e-09 3.81518035e-08 7.73191630e-07
  8.30043336e-06 1.11739769e-06 1.06304071e-07 1.18161214e-09
  9.28678583e-06 1.27551381e-09 5.96398309e-08 4.41945289e-08
  1.14834709e-02 5.86957150e-08 6.79463907e-09 3.76273420e-05
  2.38028530e-08 4.04193571e-07 1.70917738e-10 5.08988433e-06
  5.74718833e-05 1.48353308e-07 1.19810650e-09 1.19268917e-09
  2.69803208e-10 4.79320472e-10 3.48429663e-09 5.76843817e-09
  2.68716412e-03 1.76377064e-08 4.08091328e-05 8.98605967e-10
  5.07291709e-10 1.22996759e-08 